In [2]:
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime as dt
import statsmodels.formula.api as sm

In [3]:
train = pd.read_csv("data/2022_FH.csv")
train = train.dropna(subset = ["Game Prep"])
train.reset_index()

/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_39168/92136705.py:1: DtypeWarning: Columns (59,63,67) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("data/2022_FH.csv")


,index,Week,Day,W L,Game Prep,Position,Athlete,Session Name,Session,Phase Name,...,TSB HSD,Decel Sum,Acute Decel,Chronic Decel,TSB Decel,HR 4 Percent,HR 5 Percent,Percent Above 80,TRIMP,Duration (min)
0,0,1,Wednesday,W,UConn,B,Maddie Bacskai,GD-2,Training,Whole session,...,1.00,25.0,12.25,12.25,1.0,11.43%,2.59%,14.02%,1222.39,216
1,1,1,Wednesday,W,UConn,F,Bente Baekers,GD-2,Training,Whole session,...,1.00,52.0,25.25,25.25,1.0,10.25%,0.80%,11.06%,1123.52,216
2,2,1,Wednesday,W,UConn,M,Jordan Carr,GD-2,Training,Whole session,...,1.00,74.0,36.50,36.50,1.0,18.93%,1.18%,20.11%,1240.37,216
3,3,1,Wednesday,W,UConn,M,Regan Cornelius,GD-2,Training,Whole session,...,1.00,71.0,34.50,34.50,1.0,9.56%,0.20%,9.76%,1132.51,216
4,4,1,Wednesday,W,UConn,B,Shannon Daley,GD-2,Training,Whole session,...,1.00,87.0,42.75,42.75,1.0,8.42%,0.56%,8.99%,1150.49,216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16820,22398,65,Sunday,L,NCAA_UNC,M,Chloe Relford,NCAA_UNC,Game,4th quarter,...,1.17,19.0,36.18,32.85,1.1,23.59%,16.36%,39.95%,132.31,22
16821,22399,65,Sunday,L,NCAA_UNC,GK,Annabel Skubisz,NCAA_UNC,Game,4th quarter,...,1.68,1.0,4.28,6.16,0.69,8.83%,0.00%,8.83%,128.66,22
16822,22400,65,Sunday,L,NCAA_UNC,M,Lila Slattery,NCAA_UNC,Game,4th quarter,...,0.87,1.0,19.60,19.29,1.02,0.61%,0.00%,0.61%,103.11,22
16823,22401,65,Sunday,L,NCAA_UNC,M,Lauren Wadas,NCAA_UNC,Game,4th quarter,...,0.88,22.0,27.36,27.92,0.98,24.81%,13.24%,38.05%,135.05,22


In [4]:
train.columns

Index(['Week', 'Day', 'W L', 'Game Prep', 'Position', 'Athlete',
       'Session Name', 'Session', 'Phase Name', 'Duration', 'Date', 'HR Min',
       'HR AVG', 'HR Max', 'HR Min Per', 'HR Avg Per', 'HR Max Per', 'HR 1',
       'HR 2', 'HR 3', 'HR 4', 'HR 5', 'Total Distance', 'Distance/min',
       'Max Speed', 'Avg Speed', 'Sprints', 'Speed 1', 'Speed 2', 'Speed 3',
       'Speed 4', 'Speed 5', 'Decel 4', 'Decel 3', 'Decel 2', 'Decel 1',
       'Accel 1', 'Accel 2', 'Accel 3', 'Accel 4', 'Calories', 'TLScore',
       'Cardio Load', 'Recovery', 'TL Percent', 'TLScore Acute',
       'TLScore Chronic', 'TLScore TSB', 'HR Sum 4-5', 'Acute HR Sum',
       'Chronic HR Sum', 'TSB HR Sum', 'TD Percent', 'Acute TD', 'Chronic TD',
       'TSB TD', 'HIA', 'Acute HIA', 'Chronic HIA', 'TSB HIA', 'HSD',
       'Acute HSD', 'Chronic HSD', 'TSB HSD', 'Decel Sum', 'Acute Decel',
       'Chronic Decel', 'TSB Decel', 'HR 4 Percent', 'HR 5 Percent',
       'Percent Above 80', 'TRIMP', 'Duration (min)'],


**Cleaning data**
- delete all rows that aren't preparing for a game
- only use fall data (August - Dec) - regular season data
- don't include HR% because practices are varying length - actual time is more important
- filter to only include "whole session"
- filter to players who played more than 800 minutes for the 2022 season
- filter out goalies

In [5]:
#train = train[["Week", "W L", "Game Prep", "Position", "Athlete", "Session Name", "Phase Name", "Date", "HR Min", "HR AVG", "HR Max", "HR 1", "HR 2", "HR 3", "HR 4", "HR 5", "Total Distance", "Sprints", "Cardio Load", "TLScore", "Duration (min)"]]

In [6]:
train["Date"] = pd.to_datetime(train['Date'])
train.Date

0       2021-08-25
1       2021-08-25
2       2021-08-25
3       2021-08-25
4       2021-08-25
           ...    
22398   2022-11-20
22399   2022-11-20
22400   2022-11-20
22401   2022-11-20
22402   2022-11-20
Name: Date, Length: 16825, dtype: datetime64[ns]

In [7]:
#only keeping whole session, dates during season, and field player data
train = train[(train["Phase Name"] == "Whole session") & (train["Date"] >= "2022-08-01") & (train["Date"] <= "2022-12-01") & (train["Position"] != "GK")]

In [8]:
train.Athlete.unique()

array(['Amelia Albers', 'Bente Baekers', 'Jordan Carr', 'Regan Cornelius',
       'Shannon Daley', 'Sophie Dix', 'Peyton Halsey', 'Lane Herbert',
       'Greta Hinke', 'Lauren Hunter', 'Aerin Krys', 'Ana Medina Garcia',
       'Magnolia Powers', 'Chloe Relford', 'Lila Slattery',
       'Sophie Thomas', 'Lauren Wadas', 'Madeleine Zimmer',
       'Maja Zivojnovic', 'Kayla Blas', 'Alia Marshall', 'Katie Jones'],
      dtype=object)

In [9]:
#filtering out data for only players with minutes above 800 minutes of playing time over the course of the season (total possible playing time around 1600 minutes)
high_min_players = ['Bente Baekers', 'Shannon Daley', 'Peyton Halsey', 'Ana Medina Garcia', 'Chloe Relford', 'Lauren Wadas','Madeleine Zimmer','Kayla Blas','Alia Marshall']
train = train.loc[train.Athlete.isin(high_min_players),:]
train

,Week,Day,W L,Game Prep,Position,Athlete,Session Name,Session,Phase Name,Duration,...,TSB HSD,Decel Sum,Acute Decel,Chronic Decel,TSB Decel,HR 4 Percent,HR 5 Percent,Percent Above 80,TRIMP,Duration (min)
13150,52,Wednesday,NaN,Iowa,F,Bente Baekers,GD-3,Training,Whole session,3:22:48,...,1.00,101.0,32.97,36.74,0.90,23.10%,13.78%,36.88%,1259.05,203
13153,52,Wednesday,NaN,Iowa,B,Shannon Daley,GD-3,Training,Whole session,3:22:48,...,1.00,141.0,33.16,36.50,0.91,17.27%,14.99%,32.26%,1233.70,203
13155,52,Wednesday,NaN,Iowa,M,Peyton Halsey,GD-3,Training,Whole session,3:22:48,...,1.00,65.0,26.11,28.82,0.91,4.92%,8.10%,13.03%,1402.70,203
13161,52,Wednesday,NaN,Iowa,M,Ana Medina Garcia,GD-3,Training,Whole session,3:22:48,...,1.00,110.0,26.75,30.93,0.86,21.54%,7.31%,28.85%,1225.25,203
13163,52,Wednesday,NaN,Iowa,M,Chloe Relford,GD-3,Training,Whole session,3:22:48,...,1.00,148.0,34.63,35.86,0.97,14.52%,20.36%,34.89%,1233.70,203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,65,Sunday,L,NCAA_UNC,M,Alia Marshall,NCAA_UNC,Game,Whole session,2:26:00,...,0.97,109.0,30.74,31.71,0.97,37.04%,8.77%,45.81%,906.42,146
22261,65,Sunday,L,NCAA_UNC,M,Ana Medina Garcia,NCAA_UNC,Game,Whole session,2:26:00,...,0.95,111.0,29.15,27.79,1.05,32.74%,22.65%,55.39%,955.08,146
22262,65,Sunday,L,NCAA_UNC,M,Chloe Relford,NCAA_UNC,Game,Whole session,2:26:00,...,1.17,111.0,36.18,32.85,1.1,14.42%,22.09%,36.51%,900.33,146
22265,65,Sunday,L,NCAA_UNC,M,Lauren Wadas,NCAA_UNC,Game,Whole session,2:26:00,...,0.88,88.0,27.36,27.92,0.98,12.90%,8.63%,21.53%,851.67,146


In [10]:
train['W L'].isnull().sum()

40

In [11]:
# dropping all NA W_L values bc that game was a scrimmage
train = train.dropna()
train.head()

,Week,Day,W L,Game Prep,Position,Athlete,Session Name,Session,Phase Name,Duration,...,TSB HSD,Decel Sum,Acute Decel,Chronic Decel,TSB Decel,HR 4 Percent,HR 5 Percent,Percent Above 80,TRIMP,Duration (min)
13879,53,Thursday,W,Maine,F,Bente Baekers,GD-3,Training,Whole session,2:02:28,...,0.94,80.0,32.50,32.66,0.99,22.77%,7.09%,29.86%,699.08,122
13880,53,Thursday,W,Maine,B,Kayla Blas,GD-3,Training,Whole session,2:02:28,...,1.24,62.0,25.67,29.48,0.87,18.02%,0.00%,18.02%,653.16,122
13883,53,Thursday,W,Maine,B,Shannon Daley,GD-3,Training,Whole session,2:02:28,...,0.91,99.0,34.18,34.54,0.99,20.79%,6.76%,27.56%,683.77,122
13885,53,Thursday,W,Maine,M,Peyton Halsey,GD-3,Training,Whole session,2:02:28,...,1.14,65.0,29.62,28.14,1.05,14.02%,26.29%,40.31%,770.52,122
13892,53,Thursday,W,Maine,M,Alia Marshall,GD-3,Training,Whole session,2:02:28,...,1.07,57.0,25.48,26.50,0.96,16.59%,0.14%,16.73%,648.05,122


In [12]:
#adding a binary column for W and Loss
train.columns = train.columns.str.replace(' ', '_')
train['W_L_binary'] = train['W_L'].apply(lambda x: 0 if x=='L' else 1)
train.head()

/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_39168/1939409511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['W_L_binary'] = train['W_L'].apply(lambda x: 0 if x=='L' else 1)


,Week,Day,W_L,Game_Prep,Position,Athlete,Session_Name,Session,Phase_Name,Duration,...,Decel_Sum,Acute_Decel,Chronic_Decel,TSB_Decel,HR_4_Percent,HR_5_Percent,Percent_Above_80,TRIMP,Duration_(min),W_L_binary
13879,53,Thursday,W,Maine,F,Bente Baekers,GD-3,Training,Whole session,2:02:28,...,80.0,32.50,32.66,0.99,22.77%,7.09%,29.86%,699.08,122,1
13880,53,Thursday,W,Maine,B,Kayla Blas,GD-3,Training,Whole session,2:02:28,...,62.0,25.67,29.48,0.87,18.02%,0.00%,18.02%,653.16,122,1
13883,53,Thursday,W,Maine,B,Shannon Daley,GD-3,Training,Whole session,2:02:28,...,99.0,34.18,34.54,0.99,20.79%,6.76%,27.56%,683.77,122,1
13885,53,Thursday,W,Maine,M,Peyton Halsey,GD-3,Training,Whole session,2:02:28,...,65.0,29.62,28.14,1.05,14.02%,26.29%,40.31%,770.52,122,1
13892,53,Thursday,W,Maine,M,Alia Marshall,GD-3,Training,Whole session,2:02:28,...,57.0,25.48,26.50,0.96,16.59%,0.14%,16.73%,648.05,122,1


In [13]:
# Changing HR zones to minutes
train['HR_1'] = train['HR_1'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['HR_2'] = train['HR_2'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['HR_3'] = train['HR_3'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['HR_4'] = train['HR_4'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['HR_5'] = train['HR_5'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['Acute_HR_Sum'] = train['Acute_HR_Sum'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['Chronic_HR_Sum'] = train['Chronic_HR_Sum'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_39168/4274240502.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['HR_1'] = train['HR_1'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_39168/4274240502.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['HR_2'] = train['HR_2'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_39168/4274240502.py

In [14]:
train.corrwith(train.W_L_binary).sort_values(ascending = False)

W_L_binary         1.000000
Chronic_TD         0.089316
TLScore_Chronic    0.080723
HR_AVG             0.054697
HR_Avg_Per         0.054277
Recovery           0.050656
HR_3               0.044949
Chronic_HR_Sum     0.026896
Decel_2            0.023906
Acute_HIA          0.021393
HR_Max_Per         0.020642
HR_Max             0.019570
Chronic_HIA        0.015943
Speed_5            0.014181
TLScore_Acute      0.012378
Accel_4            0.010027
TLScore            0.007662
HIA                0.006607
Accel_3            0.005298
Accel_2            0.003867
HR_4               0.003810
Sprints            0.003068
Accel_1           -0.000331
HSD               -0.000598
Cardio_Load       -0.001973
Max_Speed         -0.006403
HR_5              -0.006763
HR_2              -0.007620
Speed_4           -0.009407
Calories          -0.010086
Decel_3           -0.015878
Acute_HR_Sum      -0.021777
Decel_Sum         -0.022557
TRIMP             -0.026036
Speed_3           -0.026278
HR_Min_Per        -0

In [15]:
## Below here I am playing around with shifting the dataframe. 
# these two lines show that there is an unequal number of observations for each day. 
# I am going to try to fix this by grouping by date and taking the positional avgs for each day. after this, 
# we should be able to shift by date

In [16]:
train.Athlete.value_counts()
train.Date

13879   2022-08-25
13880   2022-08-25
13883   2022-08-25
13885   2022-08-25
13892   2022-08-25
           ...    
22260   2022-11-20
22261   2022-11-20
22262   2022-11-20
22265   2022-11-20
22266   2022-11-20
Name: Date, Length: 674, dtype: datetime64[ns]

# essentially, we need to make a for loop that assess whether each date is in the dataframe in our date range bc off days wont have data and we need to account for that. If the date is not there, we need to insert 3 rows w F M and B data. When creating the new row of data, you need the date and whether it was a W_L at the following game. Otherwise, every observation can be 0.

steps
1. assess if the date appears in train (use TF df)
2. if True, continue (just go to beginning of for loop)
3. if False, add a new row w the Date


In [17]:
len(pd.date_range('08/25/2022','11/20/2022'))

88

In [18]:
len(train.Date.unique())

69

In [19]:
date_range = pd.date_range('08/25/2022','11/20/2022')
#date_range = pd.DataFrame(date_range)
#TF_date_range = 
TF_date_range = pd.DataFrame(date_range.isin(train.Date))
TF_date_range.value_counts()
#date_range

True     69
False    19
dtype: int64

In [20]:
#len(train.Date)
#date_range.isin(train.Date)
train = train.loc[train.Athlete.isin(high_min_players),:]

In [21]:
#train.columns

In [22]:
#T_F = pd.DataFrame(date_range.isin(train.Date))
#T_F.iloc[1,0]
date_range = pd.date_range('08/25/2022','11/20/2022')
date_range = pd.DataFrame(date_range)
pd.DataFrame(date_range.isin(train.Date))

,0
0,False
1,False
2,False
3,False
4,False
...,...
83,False
84,False
85,False
86,False


In [23]:
date_range = pd.date_range('08/25/2022','11/20/2022')
#date_range = pd.DataFrame(date_range)
#TF_date_range = 
TF_date_range = pd.DataFrame(date_range.isin(train.Date))
TF_date_range.value_counts()


#date_range = pd.date_range('08/25/2022','11/20/2022')
#date_range = pd.DataFrame(date_range)
#T_F = pd.DataFrame(date_range.isin(train.Date))

for i in range(len(date_range)):
    a = pd.DataFrame(date_range)
    #print(TF_date_range.iloc[i,0])
    
    if TF_date_range.iloc[i,0] == False:
        
        new_rows = [{'Week':0, 'Day':None, 'W_L':None, 'Game_Prep': None, 'Position':'F', 'Athlete':None,
             'Session_Name':None, 'Session':None, 'Phase_Name':None, 'Duration':0, 'Date':a.iloc[i,0], 'HR_Min':0,
             'HR_AVG':0, 'HR_Max':0, 'HR_Min_Per':0, 'HR_Avg_Per':0, 'HR_Max_Per':0, 'HR_1':0,
             'HR_2':0, 'HR_3':0, 'HR_4':0, 'HR_5':0, 'Total_Distance':0, 'Distance/min':0,
             'Max_Speed':0, 'Avg_Speed':0, 'Sprints':0, 'Speed_1':0, 'Speed_2':0, 'Speed_3':0,
             'Speed_4':0, 'Speed_5':0, 'Decel_4':0, 'Decel_3':0, 'Decel_2':0, 'Decel_1':0,
             'Accel_1':0, 'Accel_2':0, 'Accel_3':0, 'Accel_4':0, 'Calories':0, 'TLScore':0,
             'Cardio_Load':0, 'Recovery':0, 'TL_Percent':0, 'TLScore_Acute':0,
             'TLScore_Chronic':0, 'TLScore_TSB':0, 'HR_Sum_4-5':0, 'Acute_HR_Sum':0,
             'Chronic_HR_Sum':0, 'TSB_HR_Sum':0, 'TD_Percent':0, 'Acute_TD':0, 'Chronic_TD':0,
             'TSB_TD':0, 'HIA':0, 'Acute_HIA':0, 'Chronic_HIA':0, 'TSB_HIA':0, 'HSD':0,
             'Acute_HSD':0, 'Chronic_HSD':0, 'TSB_HSD':0, 'Decel_Sum':0, 'Acute_Decel':0,
             'Chronic_Decel':0, 'TSB_Decel':0, 'HR_4_Percent':None, 'HR_5_Percent':None,
             'Percent_Above_80':None, 'TRIMP':0, 'Duration_(min)':0, 'W_L_binary':0},
                    {'Week':0, 'Day':None, 'W_L':None, 'Game_Prep': None, 'Position':'B', 'Athlete':None,
             'Session_Name':None, 'Session':None, 'Phase_Name':None, 'Duration':0, 'Date':a.iloc[i,0], 'HR_Min':0,
             'HR_AVG':0, 'HR_Max':0, 'HR_Min_Per':0, 'HR_Avg_Per':0, 'HR_Max_Per':0, 'HR_1':0,
             'HR_2':0, 'HR_3':0, 'HR_4':0, 'HR_5':0, 'Total_Distance':0, 'Distance/min':0,
             'Max_Speed':0, 'Avg_Speed':0, 'Sprints':0, 'Speed_1':0, 'Speed_2':0, 'Speed_3':0,
             'Speed_4':0, 'Speed_5':0, 'Decel_4':0, 'Decel_3':0, 'Decel_2':0, 'Decel_1':0,
             'Accel_1':0, 'Accel_2':0, 'Accel_3':0, 'Accel_4':0, 'Calories':0, 'TLScore':0,
             'Cardio_Load':0, 'Recovery':0, 'TL_Percent':0, 'TLScore_Acute':0,
             'TLScore_Chronic':0, 'TLScore_TSB':0, 'HR_Sum_4-5':0, 'Acute_HR_Sum':0,
             'Chronic_HR_Sum':0, 'TSB_HR_Sum':0, 'TD_Percent':0, 'Acute_TD':0, 'Chronic_TD':0,
             'TSB_TD':0, 'HIA':0, 'Acute_HIA':0, 'Chronic_HIA':0, 'TSB_HIA':0, 'HSD':0,
             'Acute_HSD':0, 'Chronic_HSD':0, 'TSB_HSD':0, 'Decel_Sum':0, 'Acute_Decel':0,
             'Chronic_Decel':0, 'TSB_Decel':0, 'HR_4_Percent':None, 'HR_5_Percent':None,
             'Percent_Above_80':None, 'TRIMP':0, 'Duration_(min)':0, 'W_L_binary':0},
                    {'Week':0, 'Day':None, 'W_L':None, 'Game_Prep': None, 'Position':'M', 'Athlete':None,
             'Session_Name':None, 'Session':None, 'Phase_Name':None, 'Duration':0, 'Date':a.iloc[i,0], 'HR_Min':0,
             'HR_AVG':0, 'HR_Max':0, 'HR_Min_Per':0, 'HR_Avg_Per':0, 'HR_Max_Per':0, 'HR_1':0,
             'HR_2':0, 'HR_3':0, 'HR_4':0, 'HR_5':0, 'Total_Distance':0, 'Distance/min':0,
             'Max_Speed':0, 'Avg_Speed':0, 'Sprints':0, 'Speed_1':0, 'Speed_2':0, 'Speed_3':0,
             'Speed_4':0, 'Speed_5':0, 'Decel_4':0, 'Decel_3':0, 'Decel_2':0, 'Decel_1':0,
             'Accel_1':0, 'Accel_2':0, 'Accel_3':0, 'Accel_4':0, 'Calories':0, 'TLScore':0,
             'Cardio_Load':0, 'Recovery':0, 'TL_Percent':0, 'TLScore_Acute':0,
             'TLScore_Chronic':0, 'TLScore_TSB':0, 'HR_Sum_4-5':0, 'Acute_HR_Sum':0,
             'Chronic_HR_Sum':0, 'TSB_HR_Sum':0, 'TD_Percent':0, 'Acute_TD':0, 'Chronic_TD':0,
             'TSB_TD':0, 'HIA':0, 'Acute_HIA':0, 'Chronic_HIA':0, 'TSB_HIA':0, 'HSD':0,
             'Acute_HSD':0, 'Chronic_HSD':0, 'TSB_HSD':0, 'Decel_Sum':0, 'Acute_Decel':0,
             'Chronic_Decel':0, 'TSB_Decel':0, 'HR_4_Percent':None, 'HR_5_Percent':None,
             'Percent_Above_80':None, 'TRIMP':0, 'Duration_(min)':0, 'W_L_binary':0}]
        train = train.append(new_rows, ignore_index=True)
        
train

/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_39168/455957058.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(new_rows, ignore_index=True)
/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_39168/455957058.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(new_rows, ignore_index=True)
/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_39168/455957058.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(new_rows, ignore_index=True)
/var/folders/gk/ct13sb9j1z7212s1w_gfshdm0000gn/T/ipykernel_39168/455957058.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

,Week,Day,W_L,Game_Prep,Position,Athlete,Session_Name,Session,Phase_Name,Duration,...,Decel_Sum,Acute_Decel,Chronic_Decel,TSB_Decel,HR_4_Percent,HR_5_Percent,Percent_Above_80,TRIMP,Duration_(min),W_L_binary
0,53,Thursday,W,Maine,F,Bente Baekers,GD-3,Training,Whole session,2:02:28,...,80.0,32.50,32.66,0.99,22.77%,7.09%,29.86%,699.08,122,1
1,53,Thursday,W,Maine,B,Kayla Blas,GD-3,Training,Whole session,2:02:28,...,62.0,25.67,29.48,0.87,18.02%,0.00%,18.02%,653.16,122,1
2,53,Thursday,W,Maine,B,Shannon Daley,GD-3,Training,Whole session,2:02:28,...,99.0,34.18,34.54,0.99,20.79%,6.76%,27.56%,683.77,122,1
3,53,Thursday,W,Maine,M,Peyton Halsey,GD-3,Training,Whole session,2:02:28,...,65.0,29.62,28.14,1.05,14.02%,26.29%,40.31%,770.52,122,1
4,53,Thursday,W,Maine,M,Alia Marshall,GD-3,Training,Whole session,2:02:28,...,57.0,25.48,26.50,0.96,16.59%,0.14%,16.73%,648.05,122,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,0,None,None,None,B,None,None,None,None,0,...,0.0,0.00,0.00,0,None,None,None,0.00,0,0
727,0,None,None,None,M,None,None,None,None,0,...,0.0,0.00,0.00,0,None,None,None,0.00,0,0
728,0,None,None,None,F,None,None,None,None,0,...,0.0,0.00,0.00,0,None,None,None,0.00,0,0
729,0,None,None,None,B,None,None,None,None,0,...,0.0,0.00,0.00,0,None,None,None,0.00,0,0


## sort in order of date, then use n/a to fill in

In [24]:
train.sort_values(by='Date', inplace=True) 
train['W_L'].fillna(method='ffill', inplace=True)
train['W_L'].isna().sum()
#train['W_L_binary'] = train['W_L'].apply(lambda x: 0 if x=='L' else 1)
train.corrwith(train.W_L_binary).sort_values(ascending = False)

W_L_binary         1.000000
TLScore_Chronic    0.086284
Chronic_TD         0.075295
HR_AVG             0.072445
HR_Avg_Per         0.072317
HR_3               0.063470
Chronic_HR_Sum     0.060087
HR_Max_Per         0.059672
Recovery           0.059574
HR_Max             0.059298
Acute_HIA          0.058933
Chronic_HIA        0.057719
TLScore_Acute      0.053843
Decel_2            0.050194
Chronic_Decel      0.042045
Acute_TD           0.035955
Acute_Decel        0.035569
Max_Speed          0.033941
Accel_1            0.032235
Accel_4            0.031671
Accel_2            0.031380
HIA                0.031253
Week               0.030834
TLScore            0.029710
Accel_3            0.029631
Speed_5            0.026633
Acute_HR_Sum       0.025240
HR_2               0.023421
Cardio_Load        0.023264
TLScore_TSB        0.022454
HR_4               0.020791
Calories           0.020769
Sprints            0.019944
TSB_TD             0.017991
HSD                0.016044
TRIMP              0

In [25]:
train.Date.value_counts()

2022-08-25    18
2022-09-27    18
2022-09-13    18
2022-09-07    17
2022-09-28    17
              ..
2022-10-22     3
2022-09-20     3
2022-09-19     3
2022-10-17     3
2022-09-12     3
Name: Date, Length: 88, dtype: int64

In [26]:
date_range = pd.date_range('08/25/2022','11/20/2022')
date_range = pd.DataFrame(date_range)
TF_date_range = date_range.isin(train.Date)
date_range

,0
0,2022-08-25
1,2022-08-26
2,2022-08-27
3,2022-08-28
4,2022-08-29
...,...
83,2022-11-16
84,2022-11-17
85,2022-11-18
86,2022-11-19


In [27]:
#i = 0
date_range = pd.date_range('08/25/2022','11/20/2022')
#train.loc[train.Date.isin(train.Date)]
date_range.isin(train.Date)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [ ]:
gtrain_D_and_Ath = train.groupby(['Date','Athlete'])


In [47]:
gtrain_D_and_Ath = train.groupby(['Date','Athlete'])
#gtrain_D_and_Ath.mean()
gtrain_Athlete = gtrain_D_and_Ath.mean()
gtrain_Athlete
#gtrain_D_and_Ath.shift()

Week  HR_Min  HR_AVG  HR_Max  HR_Min_Per  \
Date       Athlete                                                       
2022-08-25 Alia Marshall      53.0    71.5   127.0   172.0        36.0   
           Ana Medina Garcia  53.0    78.5   120.5   192.5        39.5   
           Bente Baekers      53.0    82.0   124.0   200.5        41.0   
           Chloe Relford      53.0    52.5   133.5   214.5        26.5   
           Kayla Blas         53.0    80.5   126.0   171.0        41.0   
...                            ...     ...     ...     ...         ...   
2022-11-20 Kayla Blas         65.0    92.0   142.0   180.0        47.0   
           Lauren Wadas       65.0    19.0   140.0   228.0        10.0   
           Madeleine Zimmer   65.0    26.0   128.0   224.0        13.0   
           Peyton Halsey      65.0    41.0   153.0   212.0        21.0   
           Shannon Daley      65.0    77.0   150.0   208.0        37.0   

                              HR_Avg_Per  HR_Max_Per  HR_1  HR_2  HR_3  ...  \
Date       Athlete                                                      ...   
2022-08-25 Alia Marshall            64.0        86.5  27.5  17.5  20.0  ...   
           Ana Medina Garcia        60.5        96.5  32.0  16.0   8.5  ...   
           Bente Baekers            62.5       101.5  34.5  20.5   8.0  ...   
           Chloe Relford            67.0       107.5  21.5  19.0  14.5  ...   
           Kayla Blas               64.0        86.5  31.0  19.0  17.5  ...   
...                                  ...         ...   ...   ...   ...  ...   
2022-11-20 Kayla Blas               72.0        91.0  27.0  26.0  25.0  ...   
           Lauren Wadas             70.0       114.0  22.0  31.0  24.0  ...   
           Madeleine Zimmer         64.0       112.0  22.0  34.0  23.0  ...   
           Peyton Halsey            77.0       107.0  13.0  33.0  26.0  ...   
           Shannon Daley            73.0       101.0  17.0  22.0  21.0  ...   

                              Chronic_HIA     HSD  Acute_HSD  Chronic_HSD  \
Date       Athlete                                                          
2022-08-25 Alia Marshall            34.58   310.5     234.52       219.66   
           Ana Medina Garcia        31.91   386.5     298.71       239.76   
           Bente Baekers            38.46   350.0     260.61       276.11   
           Chloe Relford            42.59   401.5     342.86       326.07   
           Kayla Blas               28.86   398.0     268.44       217.13   
...                                   ...     ...        ...          ...   
2022-11-20 Kayla Blas               23.71  1169.0     268.32       269.03   
           Lauren Wadas             26.31  1395.0     318.82       363.87   
           Madeleine Zimmer         39.95  1947.0     573.36       527.48   
           Peyton Halsey            37.41  1658.0     381.64       377.60   
           Shannon Daley            39.70  1731.0     474.61       473.10   

                              Decel_Sum  Acute_Decel  Chronic_Decel    TRIMP  \
Date       Athlete                                                             
2022-08-25 Alia Marshall           31.0        25.48          26.50  476.820   
           Ana Medina Garcia       36.0        31.11          28.61  465.225   
           Bente Baekers           46.5        32.50          32.66  483.085   
           Chloe Relford           42.0        39.04          36.39  511.340   
           Kayla Blas              36.0        25.67          29.48  475.770   
...                                 ...          ...            ...      ...   
2022-11-20 Kayla Blas              99.0        24.43          25.92  863.830   
           Lauren Wadas            88.0        27.36          27.92  851.670   
           Madeleine Zimmer       117.0        36.77          36.49  778.670   
           Peyton Halsey          115.0        31.93          31.76  930.750   
           Shannon Daley          147.0        41.82          40.24  912.500 

In [30]:
gtrain_Athlete.corrwith(gtrain_Athlete.W_L_binary).sort_values(ascending = False)

W_L_binary         1.000000
Chronic_TD         0.095602
TLScore_Chronic    0.084233
HR_3               0.073668
HR_AVG             0.072942
HR_Avg_Per         0.072842
Recovery           0.060157
Decel_2            0.045792
Chronic_HR_Sum     0.033009
TLScore            0.028057
HR_Max_Per         0.027895
Accel_1            0.027109
HR_Max             0.026591
HIA                0.024188
Accel_3            0.023949
Acute_HIA          0.022545
HR_4               0.020671
Accel_2            0.020326
Chronic_HIA        0.020010
Cardio_Load        0.018632
Accel_4            0.018381
Speed_5            0.017849
TLScore_Acute      0.017161
HR_2               0.016948
Calories           0.011878
Max_Speed          0.006547
HSD                0.006071
Sprints            0.005038
HR_5               0.002904
Speed_4           -0.001493
Decel_3           -0.002355
TRIMP             -0.004836
Decel_1           -0.007122
Decel_Sum         -0.011472
Speed_3           -0.018037
Avg_Speed         -0

In [28]:
grouped_train = train.groupby(['Date','Position'])
gtrain_date_pos = grouped_train.mean()
gtrain_date_pos

Week     HR_Min      HR_AVG      HR_Max  HR_Min_Per  \
Date       Position                                                        
2022-08-25 B         53.0  74.500000  123.500000  179.000000   37.000000   
           F         53.0  82.000000  124.000000  200.500000   41.000000   
           M         53.0  59.583333  129.166667  197.000000   30.000000   
2022-08-26 B         53.0  61.000000  115.000000  178.000000   30.500000   
           F         53.0  78.000000  122.000000  183.000000   39.000000   
...                   ...        ...         ...         ...         ...   
2022-11-19 F         65.0  70.000000  109.000000  190.000000   35.000000   
           M         65.0  43.166667  108.666667  199.000000   21.833333   
2022-11-20 B         65.0  84.500000  146.000000  194.000000   42.000000   
           F         65.0  58.000000  154.000000  202.000000   29.000000   
           M         65.0  42.666667  145.833333  206.166667   21.500000   

                     HR_Avg_Per  HR_Max_Per       HR_1       HR_2       HR_3  \
Date       Position                                                            
2022-08-25 B          61.250000   88.750000  27.500000  14.750000  13.500000   
           F          62.500000  101.500000  34.500000  20.500000   8.000000   
           M          64.916667   98.833333  23.333333  19.166667  13.416667   
2022-08-26 B          57.000000   88.000000  34.000000  14.500000  15.000000   
           F          62.000000   92.000000  43.000000  16.000000  13.000000   
...                         ...         ...        ...        ...        ...   
2022-11-19 F          55.000000   96.000000  28.000000  15.000000  10.000000   
           M          54.333333   99.833333  22.500000  12.333333   7.500000   
2022-11-20 B          72.500000   96.000000  22.000000  24.000000  23.000000   
           F          78.000000  102.000000  10.000000  31.000000  24.000000   
           M          73.166667  103.500000  18.833333  28.833333  22.166667   

                     ...  Chronic_HIA          HSD   Acute_HSD  Chronic_HSD  \
Date       Position  ...                                                      
2022-08-25 B         ...    32.725000   405.750000  272.650000   260.145000   
           F         ...    38.460000   350.000000  260.610000   276.110000   
           M         ...    37.215000   410.250000  324.761667   293.058333   
2022-08-26 B         ...    32.435000   266.500000  280.890000   256.410000   
           F         ...    38.120000   238.000000  252.040000   270.770000   
...                  ...          ...          ...         ...          ...   
2022-11-19 F         ...    32.880000    76.000000  301.140000   327.470000   
           M         ...    34.893333   162.500000  364.363333   375.616667   
2022-11-20 B         ...    31.705000  1450.000000  371.465000   371.065000   
           F         ...    32.530000  1043.000000  328.500000   331.350000   
           M         ...    34.801667  1611.833333  396.076667   387.156667   

                      Decel_Sum  Acute_Decel  Chronic_Decel       TRIMP  \
Date       Position                                                       
2022-08-25 B          43.000000    29.925000      32.010000  473.797500   
           F          46.500000    32.500000      32.660000  483.085000   
           M          38.000000    31.476667      31.048333  495.518333   
2022-08-26 B          37.500000    30.255000      31.930000  488.430000   
           F          26.000000    30.180000      32.550000  518.160000   
...                         ...          ...            ...         ...   
2022-11-19 F          24.000000    26.210000      27.830000  418.970000   
           M          32.166667    30.960000      31.470000  417.688333   
2022-11-20 B         123.000000    33.125000      33.080000  888.165000   
           F          86.000000    27.500000      27.810000  936.830000   
           M         108.500000    32.021667      31.420000  88

In [29]:
gtrain_date_pos.corrwith(gtrain_date_pos.W_L_binary).sort_values(ascending = False)

W_L_binary         1.000000
HR_3               0.101802
TLScore_Chronic    0.093996
HR_2               0.090491
Chronic_TD         0.089169
HR_Avg_Per         0.085264
HR_AVG             0.085176
Chronic_HIA        0.080359
Chronic_HR_Sum     0.079762
Decel_2            0.079227
HR_Max_Per         0.079216
HR_Max             0.078969
Acute_HIA          0.076082
Recovery           0.074954
Chronic_Decel      0.074293
Max_Speed          0.073634
Accel_1            0.071173
Accel_2            0.068289
TLScore_Acute      0.066190
TLScore            0.064591
Calories           0.063408
Acute_Decel        0.063406
Accel_4            0.063214
HIA                0.062928
TRIMP              0.061499
Accel_3            0.061280
Cardio_Load        0.060547
Acute_TD           0.060395
Decel_1            0.059789
Week               0.058981
Speed_5            0.056299
Decel_3            0.056089
Duration_(min)     0.052351
Avg_Speed          0.051380
Sprints            0.050763
HSD                0

In [52]:
gtrain_pos = train.groupby('Position')
a = gtrain_pos.corr()
a.loc[:,'W_L_binary'].sort_values(ascending = False)

#ufo.loc[:, ['City', 'State']]

#gtrain_pos.corrwith(gtrain_pos.W_L_binary)

#a.loc[:,'W_L_binary']
#a.iloc[54,:].sort_values(ascending = False)

Position                 
M         W_L_binary         1.000000
B         W_L_binary         1.000000
F         W_L_binary         1.000000
          TLScore_Chronic    0.117747
          HR_2               0.113430
                               ...   
M         HR_1              -0.063860
B         HR_4              -0.065378
M         Chronic_HSD       -0.067164
B         Acute_HR_Sum      -0.070599
          TSB_HR_Sum        -0.095601
Name: W_L_binary, Length: 165, dtype: float64

In [30]:
#print(list(gtrain_date_pos.index.value_counts()))
#a = gtrain_date_pos.reset_index(inplace=True)
gtrain_date_pos.Date.value_counts(ascending = True).head(30)

AttributeError: 'DataFrame' object has no attribute 'Date'

## once we r here

1. run corr, decide which vars to keep and what to drop
2. shift the data by 4 days (have to shift by index bc some of the dates to not have data for all 3 positions)
3. run the logit regression 

In [31]:
%pylab inline
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
plt.rcParams['figure.figsize'] = [9, 5]

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [32]:
# A few helper functions
import numpy.ma as ma
from scipy.stats.stats import pearsonr, normaltest
from scipy.spatial.distance import correlation
def build_model(features):
  X=sm.add_constant(df[features])
  y=df['power']
  model = sm.OLS(y,X, missing='drop').fit()
  predictions = model.predict(X) 
  display(model.summary()) 
  res=y-predictions
  return res 


def plt_residual(res):
  plt.plot(range(len(res)), res) 
  plt.ylabel('Residual')
  plt.xlabel("Hour")

def plt_residual_lag(res, nlag):
  x=res.values
  y=res.shift(nlag).values
  sns.kdeplot(x,y=y,color='blue',shade=True )
  plt.xlabel('res')
  plt.ylabel("res-lag-{}".format(nlag))
  rho,p=corrcoef(x,y)
  plt.title("n_lag={} hours, correlation={:f}".format(nlag, rho))
  
def plt_acf(res):
  plt.rcParams['figure.figsize'] = [18, 5]
  acorr = sm.tsa.acf(res.dropna(), nlags = len(res.dropna())-1)
  fig, (ax1, ax2) = plt.subplots(1, 2)
  ax1.plot(acorr)
  ax1.set_ylabel('corr')
  ax1.set_xlabel('n_lag')
  ax1.set_title('Auto Correlation')
  ax2.plot(acorr[:4*7*24])
  ax2.set_ylabel('corr')
  ax2.set_xlabel('n_lag')
  ax2.set_title('Auto Correlation (4-week zoomed in) ')
  plt.show()
  pd.set_option('display.max_columns', None)
  adf=pd.DataFrame(np.round(acorr[:30*24],2).reshape([30, 24] ))
  adf.index.name='day'
  display(adf)
  plt.rcParams['figure.figsize'] = [9, 5]

def corrcoef(x,y):
    a,b=ma.masked_invalid(x),ma.masked_invalid(y)
    msk = (~a.mask & ~b.mask)
    return pearsonr(x[msk],y[msk])[0], normaltest(res, nan_policy='omit')[1]

In [34]:
gtrain_Athlete.columns

Index(['Week', 'HR_Min', 'HR_AVG', 'HR_Max', 'HR_Min_Per', 'HR_Avg_Per',
       'HR_Max_Per', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5', 'Total_Distance',
       'Distance/min', 'Max_Speed', 'Avg_Speed', 'Sprints', 'Speed_1',
       'Speed_2', 'Speed_3', 'Speed_4', 'Speed_5', 'Decel_4', 'Decel_3',
       'Decel_2', 'Decel_1', 'Accel_1', 'Accel_2', 'Accel_3', 'Accel_4',
       'Calories', 'TLScore', 'Cardio_Load', 'Recovery', 'TLScore_Acute',
       'TLScore_Chronic', 'TLScore_TSB', 'Acute_HR_Sum', 'Chronic_HR_Sum',
       'TSB_HR_Sum', 'Acute_TD', 'Chronic_TD', 'TSB_TD', 'HIA', 'Acute_HIA',
       'Chronic_HIA', 'HSD', 'Acute_HSD', 'Chronic_HSD', 'Decel_Sum',
       'Acute_Decel', 'Chronic_Decel', 'TRIMP', 'Duration_(min)',
       'W_L_binary'],
      dtype='object')

In [39]:
#df=pd.read_csv("./Datasets/Toronto_power_demand.csv", parse_dates=['Date'], index_col=0)
#df['temperature']=df['temperature'].shift(24*1)
#df.tail()
gtrain_Athlete.groupby(['Date','Athlete'])
gtrain_Athlete
#train.groupby([])['value'].shift()

Week  HR_Min  HR_AVG  HR_Max  HR_Min_Per  \
Date       Athlete                                                       
2022-08-25 Alia Marshall      53.0    71.5   127.0   172.0        36.0   
           Ana Medina Garcia  53.0    78.5   120.5   192.5        39.5   
           Bente Baekers      53.0    82.0   124.0   200.5        41.0   
           Chloe Relford      53.0    52.5   133.5   214.5        26.5   
           Kayla Blas         53.0    80.5   126.0   171.0        41.0   
...                            ...     ...     ...     ...         ...   
2022-11-20 Kayla Blas         65.0    92.0   142.0   180.0        47.0   
           Lauren Wadas       65.0    19.0   140.0   228.0        10.0   
           Madeleine Zimmer   65.0    26.0   128.0   224.0        13.0   
           Peyton Halsey      65.0    41.0   153.0   212.0        21.0   
           Shannon Daley      65.0    77.0   150.0   208.0        37.0   

                              HR_Avg_Per  HR_Max_Per  HR_1  HR_2  HR_3  ...  \
Date       Athlete                                                      ...   
2022-08-25 Alia Marshall            64.0        86.5  27.5  17.5  20.0  ...   
           Ana Medina Garcia        60.5        96.5  32.0  16.0   8.5  ...   
           Bente Baekers            62.5       101.5  34.5  20.5   8.0  ...   
           Chloe Relford            67.0       107.5  21.5  19.0  14.5  ...   
           Kayla Blas               64.0        86.5  31.0  19.0  17.5  ...   
...                                  ...         ...   ...   ...   ...  ...   
2022-11-20 Kayla Blas               72.0        91.0  27.0  26.0  25.0  ...   
           Lauren Wadas             70.0       114.0  22.0  31.0  24.0  ...   
           Madeleine Zimmer         64.0       112.0  22.0  34.0  23.0  ...   
           Peyton Halsey            77.0       107.0  13.0  33.0  26.0  ...   
           Shannon Daley            73.0       101.0  17.0  22.0  21.0  ...   

                              Chronic_HIA     HSD  Acute_HSD  Chronic_HSD  \
Date       Athlete                                                          
2022-08-25 Alia Marshall            34.58   310.5     234.52       219.66   
           Ana Medina Garcia        31.91   386.5     298.71       239.76   
           Bente Baekers            38.46   350.0     260.61       276.11   
           Chloe Relford            42.59   401.5     342.86       326.07   
           Kayla Blas               28.86   398.0     268.44       217.13   
...                                   ...     ...        ...          ...   
2022-11-20 Kayla Blas               23.71  1169.0     268.32       269.03   
           Lauren Wadas             26.31  1395.0     318.82       363.87   
           Madeleine Zimmer         39.95  1947.0     573.36       527.48   
           Peyton Halsey            37.41  1658.0     381.64       377.60   
           Shannon Daley            39.70  1731.0     474.61       473.10   

                              Decel_Sum  Acute_Decel  Chronic_Decel    TRIMP  \
Date       Athlete                                                             
2022-08-25 Alia Marshall           31.0        25.48          26.50  476.820   
           Ana Medina Garcia       36.0        31.11          28.61  465.225   
           Bente Baekers           46.5        32.50          32.66  483.085   
           Chloe Relford           42.0        39.04          36.39  511.340   
           Kayla Blas              36.0        25.67          29.48  475.770   
...                                 ...          ...            ...      ...   
2022-11-20 Kayla Blas              99.0        24.43          25.92  863.830   
           Lauren Wadas            88.0        27.36          27.92  851.670   
           Madeleine Zimmer       117.0        36.77          36.49  778.670   
           Peyton Halsey          115.0        31.93          31.76  930.750   
           Shannon Daley          147.0        41.82          40.24  912.500 